In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
# Pkg.add("PyPlot")
# Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random
using Plots
# using PyPlot
# using PyCall

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [25]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.90 
#alphas = [0.90, 0.95, 0.99]
#make beta a list of values
#betas = [0.1, 1, 100, 1000, 10000, 1000000, 10000000, 10000000000000000000000000000000000000]
#beta = 0.1
betas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
objectiv_values = []
exp_profits = []
CVaR_values = []

Any[]

In [43]:
# create a dataframe to store the profits per scenario for each beta
profits_df = DataFrame()

# add a first column to the dataframe for beta
profits_df[!, "beta"] = zeros(length(betas))

# add columns to the dataframe for the profits per scenario
for i in 1:W
    profits_df[!, Symbol("profit_scenario_" * string(i))] = zeros(length(betas))
end

profits_df

Row,beta,profit_scenario_1,profit_scenario_2,profit_scenario_3,profit_scenario_4,profit_scenario_5,profit_scenario_6,profit_scenario_7,profit_scenario_8,profit_scenario_9,profit_scenario_10,profit_scenario_11,profit_scenario_12,profit_scenario_13,profit_scenario_14,profit_scenario_15,profit_scenario_16,profit_scenario_17,profit_scenario_18,profit_scenario_19,profit_scenario_20,profit_scenario_21,profit_scenario_22,profit_scenario_23,profit_scenario_24,profit_scenario_25,profit_scenario_26,profit_scenario_27,profit_scenario_28,profit_scenario_29,profit_scenario_30,profit_scenario_31,profit_scenario_32,profit_scenario_33,profit_scenario_34,profit_scenario_35,profit_scenario_36,profit_scenario_37,profit_scenario_38,profit_scenario_39,profit_scenario_40,profit_scenario_41,profit_scenario_42,profit_scenario_43,profit_scenario_44,profit_scenario_45,profit_scenario_46,profit_scenario_47,profit_scenario_48,profit_scenario_49,profit_scenario_50,profit_scenario_51,profit_scenario_52,profit_scenario_53,profit_scenario_54,profit_scenario_55,profit_scenario_56,profit_scenario_57,profit_scenario_58,profit_scenario_59,profit_scenario_60,profit_scenario_61,profit_scenario_62,profit_scenario_63,profit_scenario_64,profit_scenario_65,profit_scenario_66,profit_scenario_67,profit_scenario_68,profit_scenario_69,profit_scenario_70,profit_scenario_71,profit_scenario_72,profit_scenario_73,profit_scenario_74,profit_scenario_75,profit_scenario_76,profit_scenario_77,profit_scenario_78,profit_scenario_79,profit_scenario_80,profit_scenario_81,profit_scenario_82,profit_scenario_83,profit_scenario_84,profit_scenario_85,profit_scenario_86,profit_scenario_87,profit_scenario_88,profit_scenario_89,profit_scenario_90,profit_scenario_91,profit_scenario_92,profit_scenario_93,profit_scenario_94,profit_scenario_95,profit_scenario_96,profit_scenario_97,profit_scenario_98,profit_scenario_99,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [46]:
loop = 0

for beta in betas
#for alpha in alphas
    loop += 1
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    #reset the model
    

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W] >= 0)

    # Define the objective function
    @objective(model, Max, (1 - beta) * sum(1/W *
    (scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
    + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W)))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)


    # Solve the optimization problem
    optimize!(model)

    exp_profit = sum(1/W .*
    (scenarios[i][hour,"price"] * value.(p_DA[hour])
    + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)

    CVaR = (value.(zeta) - (1/(1-alpha)) * sum(1/W .* value.(eta[i]) for i in 1:W))

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Optimal solution found")

        println("p_DA: ", value.(p_DA))
        
        # RETURN OBJECTIVE value
        #println("Objective value: ", objective_value(model))
        #println("Expected profit: ", exp_profit)
        #println("CVaR: ", CVaR)
        push!(objectiv_values, objective_value(model))
        push!(exp_profits, exp_profit)
        push!(CVaR_values, CVaR)
    else
        println("No optimal solution found")
    end


    #save the profits of each scenario for this beta in profits_df
    profits_df[loop, "beta"] = beta
    for i in 1:W
        profits_df[loop, Symbol("profit_scenario_" * string(i))] = sum(
        (scenarios[i][hour,"price"] * value.(p_DA[hour])
        + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours)
    end

end

Optimal solution found
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.842170943040401e-14, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 199.99999999999966, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [199.99999999999997, 0.0, 0.0, 108.43225074764159, -5.115907697472721e-13, 199.99999999999974, 200.0, 0.0, 0.0, 200.0, 2.8421709430404007e-13, 0.0, 200.0, 200.0, 200.0

In [27]:
#make a dataframe with the results
results = DataFrame(betas = betas, objective_values = objectiv_values, exp_profits = exp_profits, CVaR_values = CVaR_values)

#save the results
CSV.write("1_3_results/one_price_results.csv", results)

"1_3_results/one_price_results.csv"

In [47]:
# save the profits per scenario for each beta

CSV.write("1_3_results/one_price_profits.csv", profits_df)

profits_df

Row,beta,profit_scenario_1,profit_scenario_2,profit_scenario_3,profit_scenario_4,profit_scenario_5,profit_scenario_6,profit_scenario_7,profit_scenario_8,profit_scenario_9,profit_scenario_10,profit_scenario_11,profit_scenario_12,profit_scenario_13,profit_scenario_14,profit_scenario_15,profit_scenario_16,profit_scenario_17,profit_scenario_18,profit_scenario_19,profit_scenario_20,profit_scenario_21,profit_scenario_22,profit_scenario_23,profit_scenario_24,profit_scenario_25,profit_scenario_26,profit_scenario_27,profit_scenario_28,profit_scenario_29,profit_scenario_30,profit_scenario_31,profit_scenario_32,profit_scenario_33,profit_scenario_34,profit_scenario_35,profit_scenario_36,profit_scenario_37,profit_scenario_38,profit_scenario_39,profit_scenario_40,profit_scenario_41,profit_scenario_42,profit_scenario_43,profit_scenario_44,profit_scenario_45,profit_scenario_46,profit_scenario_47,profit_scenario_48,profit_scenario_49,profit_scenario_50,profit_scenario_51,profit_scenario_52,profit_scenario_53,profit_scenario_54,profit_scenario_55,profit_scenario_56,profit_scenario_57,profit_scenario_58,profit_scenario_59,profit_scenario_60,profit_scenario_61,profit_scenario_62,profit_scenario_63,profit_scenario_64,profit_scenario_65,profit_scenario_66,profit_scenario_67,profit_scenario_68,profit_scenario_69,profit_scenario_70,profit_scenario_71,profit_scenario_72,profit_scenario_73,profit_scenario_74,profit_scenario_75,profit_scenario_76,profit_scenario_77,profit_scenario_78,profit_scenario_79,profit_scenario_80,profit_scenario_81,profit_scenario_82,profit_scenario_83,profit_scenario_84,profit_scenario_85,profit_scenario_86,profit_scenario_87,profit_scenario_88,profit_scenario_89,profit_scenario_90,profit_scenario_91,profit_scenario_92,profit_scenario_93,profit_scenario_94,profit_scenario_95,profit_scenario_96,profit_scenario_97,profit_scenario_98,profit_scenario_99,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0,2.08982e5,98701.2,3.35299e5,4.63728e5,49089.4,2.27676e5,6256.24,1.28365e5,95773.1,38887.2,60400.8,58965.9,42951.2,2.23855e5,1.31464e5,3.92557e5,1.1627e5,46685.4,1.79785e5,1.26032e5,38212.4,36487.0,1.21831e5,2.02272e5,81278.2,1.88618e5,285537.0,1.06226e5,184622.0,97984.1,2.70334e5,603857.0,1.04397e5,2.30938e5,2.84524e5,2.30589e5,98472.3,9046.91,390111.0,1.13385e5,98472.3,1.1241e5,38931.8,51668.3,1.83909e5,2.31636e5,1.71951e5,79569.9,2.58508e5,1.34003e5,1.58085e5,65405.4,2.78807e5,28107.7,56492.6,1.60572e5,1.2993e5,96773.8,1.37434e5,1.50878e5,86639.8,164479.0,88589.8,1.19254e5,2.17779e5,50274.7,1.26816e5,1.12088e5,3.6963e5,10059.5,390111.0,1.40751e5,2.01478e5,46343.6,22218.4,3.67132e5,77908.5,16371.2,1.69775e5,1.48662e5,5974.05,3.04746e5,2.45323e5,2.03236e5,4.52908e5,2.15005e5,71707.5,1.26032e5,2.15005e5,1.39623e5,126559.0,1.74358e5,78318.2,3.47736e5,1.16264e5,1.83299e5,35687.8,62986.9,2.78352e5,⋯
2,0.1,2.08982e5,98701.2,3.35299e5,4.63728e5,49089.4,2.27676e5,6256.24,1.28365e5,95773.1,38887.2,60400.8,58965.9,42951.2,2.23855e5,1.31464e5,3.92557e5,1.1627e5,46685.4,1.79785e5,1.26032e5,38212.4,36487.0,1.21831e5,2.02272e5,81278.2,1.88618e5,285537.0,1.06226e5,184622.0,97984.1,2.70334e5,603857.0,1.04397e5,2.30938e5,2.84524e5,2.30589e5,98472.3,9046.91,390111.0,1.13385e5,98472.3,1.1241e5,38931.8,516

UndefVarError: UndefVarError: `xlabel` not defined